# 장르가 나누어진 데이터를 가지고 평점 매기기

## 장르 나눠진 레시피 데이터 받아오기

In [ ]:
import pandas as pd

In [ ]:
recipe = pd.read_csv('../recipe/curry_recipe_with_genres.csv', index_col='id')
recipe.sample()

,name,composition,ingredients,servings,difficulty,thumbnail,time,orders,photo,genres
id,,,,,,,,,,
6858012,낙지볶음,든든하게,[필수재료] 낙지 3마리| 콩나물 1줌(100g)| 대파 10cm 2대| 양파 1/...,2인분,아무나,https://recipe1.ezmember.co.kr/cache/recipe/20...,30분 이내,|1. 끓는 물에 콩나물 100g을 대처 낸다.|2. 볼에 양념장을 만든다.|3. ...,|https://recipe1.ezmember.co.kr/cache/recipe/2...,ing6|ing13|ing14|ing15|ing16|ing23


In [ ]:
# 장르 결측값 ''으로 채우기
recipe['genres'] = recipe['genres'].fillna('')

In [ ]:
recipe['genres'].head()

id
6842324                      ing15|ing23
6845721      ing5|ing9|ing11|ing16|ing17
6845906    ing13|ing15|ing16|ing17|ing25
6846020     ing2|ing11|ing13|ing17|ing25
6846262                      ing24|ing25
Name: genres, dtype: object

## 페르소나 데이터 프레임 생성

### 페르소나 데이터 정제
#### 재료 텍스트 -> ing 변수 이름

In [ ]:
# ing 목록 (햄 제외)
ing1 = ['소고기', '쇠고기', '소불고기', 'LA갈비', '한우']
ing2 = ['돼지', '돼지고기', '목살', '삼겹살', '베이컨', '스팸', '소시지', '소세지', '김밥용햄', '김밥용 햄', '모듬햄', ' 햄 ', '슬라이스햄', '통조림햄', '슬라이스 햄', '통조림 햄']
ing3 = ['닭']
ing4 = ['오리고기', '훈제오리']
ing5 = ['가자미', '광어', '도미', '민어', '멸치', '고등어', '꽁치', '장어', '연어', '청어', '메기', '붕어', '빙어', '잉어', '송어', '참치', '갈치', '농어', '명태', '정어리', '삼치', '동태']
ing6 = ['꼴뚜기', '낙지', '문어', '오징어', '해삼', '해파리', '쭈꾸미', '주꾸미']
ing7 = ['가재', '왕게', '킹크랩', '랍스터', '새우', '대게', '게살', '꽃게', '게장', '게맛살', '크래미']
ing8 = ['가리비', '대합', '모시조개', '바지락', '소라', '우렁', '전복', '홍합', '꼬막', ' 굴 ', '생굴']
ing9 = ['김치', '깍두기', '묵은지']
ing10 = ['두부']
ing11 = ['달걀', '계란', '노른자', '메추리알']
ing12 = ['버섯', '표고']
ing13 = ['갓', '근대', '머위', '무청', '미나리', '양배추', '배추', '봄동', '상추', '샐러리', '시금치', '쑥갓', '깻잎', ' 냉이 ', '고사리', '콩나물', '숙주', '청경채', '치커리']
ing14 = ['두릅', '마늘쫑', '부추', '샐러리', '셀러리', '아스파라거스', '염교', '죽순', '참나리', '송송썬파', '쪽파', '대파', '다진파', '파채', '실파', ' 파 ']
ing15 = ['당근', '래디쉬', '비트', '생강', '순무', '연근', '우엉', '토란', '고구마', '감자', '옥수수', ' 무 ', '무절임', '열무']
ing16 = ['마늘', '샬롯', '양파']
ing17 = ['가지', '고추', '동아', '오이', '월과', '피망', '토마토', '호박', '파프리카']
ing18 = ['브로콜리', '아티초크', '양하', '콜리플라워']
ing19 = ['사과', '유자', '자몽', '오렌지', '라임', '레몬', '갈은배', '배즙', '배음료', ' 배 ', '갈아만든배', '곶감', '단감', ' 감 ', '귤', '한라봉']
ing20 = ['복숭아', '매실', '살구', '자두', '대추', '앵두']
ing21 = ['포도', '무화과', '석류', '크랜베리', '블루베리', '체리', '라즈베리']
ing22 = ['바나나', '파인애플', '키위', '아보카도', '망고']
ing23 = ['호두', '은행', '견과류', '콩', '아몬드', '밤', '잣', '팥']
ing24 = ['딸기', '수박', '참외']
ing25 = ['우유', '아이스크림', '치즈', '버터', '요구르트']

In [ ]:
persona = pd.read_csv('persona_data.csv')
persona.index = range(1000, 1050, 1)
persona.index.name = 'userId'

# Good, Bad(선호 불호 장르 정제한 버전) 열 추가
persona['Good'] = ''
persona['Bad'] = ''

# 필요 없는 행 삭제
persona = persona.drop(['나이/성별', '특징'], axis='columns')
persona.head()

,선호,불호,Good,Bad
userId,,,,
1000,"견과류 (호두), 어류 (고등어), 화채류 (브로콜리)",유제품 (아이스크림),,
1001,"인과류 (사과), 과채류 중 과일 (딸기), 유제품 (버터)",엽채류 (시금치),,
1002,"닭, 근채류 (무)","김치, 과채류 (피망)",,
1003,"돼지고기, 두부, 버섯",갑각류 (새우),,
1004,돼지고기,"엽채류 (시금치), 경채류 (두릅), 근채류 (당근)",,


In [ ]:
# ing 처리
def update_prefer(row):
  # NaN -> '' 처리
  if pd.isnull(row['선호']):
    row['선호'] = ''
  if pd.isnull(row['불호']):
    row['불호'] = ''

  # 선호, 불호 열에서 재료 이름 찾아내어 Good, Bad 열에 ing 변수 이름으로 저장
  for i in range(1, 26):
      ing_list = globals()[f'ing{i}']
      for ing in ing_list:
        if ing in row['선호'] and f'ing{i}|' not in row['Good']:
          row['Good'] += f'ing{i}|'
        if ing in row['불호'] and f'ing{i}|' not in row['Bad']:
          row['Bad'] += f'ing{i}|'
  return row

In [ ]:
persona = persona.apply(update_prefer, axis=1)
persona.sample()

,선호,불호,Good,Bad
userId,,,,
1029,"닭, 돼지고기","과채류 (가지), 장과류 (무화과)",ing2|ing3|,ing17|ing21|


In [ ]:
# 마지막 | 삭제
persona['Good'] = persona['Good'].str.rstrip('|')
persona['Bad'] = persona['Bad'].str.rstrip('|')

In [ ]:
# 예외 오류 데이터 확인
persona[persona.index == 1022]

,선호,불호,Good,Bad
userId,,,,
1022,"엽채류 (배추), 경채류 (부추), 구근류 (양파), 핵과류 (매실)",,ing13|ing14|ing16|ing20,


In [ ]:
# 예외 오류 데이터 수정 : ing17이 표함되는 오류
persona.loc[persona.index ==1022, 'Good'] = 'ing11|ing12|ing14|ing18'
persona[persona.index == 1022]

,선호,불호,Good,Bad
userId,,,,
1022,"엽채류 (배추), 경채류 (부추), 구근류 (양파), 핵과류 (매실)",,ing11|ing12|ing14|ing18,


## 페르소나의 선호 장르 데이터를 이용하여 레시피 평점 계산하기

In [ ]:
import re

In [ ]:
ratings = []

for persona_idx, persona_row in persona.iterrows():
  for recipe_idx, recipe_row in recipe.iterrows():
    recipe_genres = recipe_row['genres'].split('|')
    persona_good = persona_row['Good'].split('|')
    persona_bad = persona_row['Bad'].split('|')
    rating = 3
    for j in recipe_genres:
      for p in persona_good:
        if (j == p):
          rating += 0.5
      for q in persona_bad:
        if (j == q):
          rating -= 0.5
    ratings.append([recipe_idx, persona_idx, rating])
ratings = pd.DataFrame(ratings, columns=['id', 'userId', 'rating'])
ratings.set_index('id', inplace=True)

In [ ]:
ratings.sample(3)

,userId,rating
id,,
6944556,1006,3.0
6912688,1038,3.0
6865005,1037,2.5


### 장르 없는 레시피 - 유저의 평점 평균으로 채우기

In [ ]:
# 유저 평점 평균 계산
ratings_user_group = ratings.groupby('userId').mean()
ratings_user_group.sample(3)

,rating
userId,
1040,3.221792
1008,3.362417
1009,3.034292


In [ ]:
# 평점 평균으로 채우기
no_gerne_ratings = []

for recipeId in recipe[recipe['genres'] == ''].index:
  for userId in persona.index:
    no_gerne_ratings.append([recipeId, userId, ratings_user_group[userId == ratings_user_group.index]['rating'].values[0]])

no_gerne_ratings = pd.DataFrame(no_gerne_ratings, columns=['id', 'userId', 'rating'])
no_gerne_ratings.set_index('id', inplace=True)

In [ ]:
no_gerne_ratings.sample(3)

,userId,rating
id,,
6885306,1044,3.056416
6924401,1000,2.943308
6906309,1004,2.721792


### 최종 평점 - 기존 평점과 장르 없는 레시피 평점 합치기



In [ ]:
final_ratings = pd.concat([ratings, no_gerne_ratings])

In [ ]:
print(ratings.shape)
print(no_gerne_ratings.shape)
print(final_ratings.shape)

(180800, 2)
(4850, 2)
(185650, 2)


### 페르소나 평점 데이터 저장

In [ ]:
final_ratings.sample(5)

,userId,rating
id,,
6891928,1016,3.5
6907301,1011,3.0
6947763,1023,3.5
6915484,1012,3.0
6846633,1043,3.0


In [ ]:
final_ratings.to_csv('ratings.csv', encoding='utf-8-sig')